<h1 style="color:black" align="center">Многоклассовая классификация и
категориальные признаки</h1>

Ранее мы разобрались с общим подходом к решению задачи бинарной классификации, а также изучили свойства двух конкретных методов: логистической регрессии и метода опорных векторов. Теперь мы перейдём к более общей, многоклассовой постановке задачи классификации, и попытаемся понять, как можно свести её к уже известным нам методам.
Также мы обсудим методы работы с категориальными признаками и поймём, почему на таких данных чаще всего используются линейные модели.

<h1 style="color:#008B8B">1. Многоклассовая классификация</h1>

В данном разделе будем считать, что каждый объект относится к одному из $K$ классов: $\mathbb{Y} = \{1, \dots, K\}$

<h2 style="color:#008B8B">1.1 Сведение к серии бинарных задач</h2>

Мы уже подробно изучили задачу бинарной классификации, и поэтому вполне естественно попытаться свести многоклассовую задачу к набору бинарных.
Существует достаточно много способов сделать это - мы перечислим лишь два самых популярных, а об остальных можно почитать.

<h3 style="color:#008B8B">Один против всех (one-versus-all)</h3>

Допустим у нас 3 класса и для каждого класса построим классификатор, который отделяет текущий класс от остальных. При этом, каждый из классификаторов способен решить бинаруню задачу, отделить один класс от другого.

Обучим $K$ линейных классификаторов $b_1(x), \dots, b_K(x)$,
выдающих оценки принадлежности классам $1, \dots, K$ соответственно.
Например, в случае с линейными моделями эти модели будут иметь вид

$\large b_k(x) = \langle w_k, x \rangle + w_{0k}$


Сформируем выборку $X_k$ для каждого класса $k$ внутри которой объекты остаются прежними а целивая переменная принимает вид $2 [y_i = k] - 1$, тем самым объект будет является положительным если он относится к $K$ классу и отрицательным если к любому другому. То есть, мы учим классификатор отличать $k$ класс от всех остальных классов.

$\large X_k = \{(x_i, 2 [y_i = k] - 1)\}_{i = 1}^{\ell}$

Классификатор с номером $k$ будем обучать по выборке $\left(x_i, 2 [y_i = k] - 1\right)_{i = 1}^{\ell}$; иными словами, мы учим классификатор отличать $k$-й класс от всех остальных.

Итоговый классификатор будет выдавать класс, соответствующий самому уверенному из бинарных алгоритмов:

$\large a(x) = \underset{k \in \{1, \dots, K\}}{\text{argmax}} b_k(x)$ - относим объект к тому классу, где у модели наибольшая уверенность.

**Проблема:** Проблема данного подхода заключается в том, что каждый
из классификаторов $b_1(x), \dots, b_K(x)$ обучается на своей выборке, и выходы этих классификаторов могут иметь разные масштабы, из-за чего сравнивать их будет неправильно. Нормировать вектора весов, чтобы они выдавали ответы в одной и той же шкале, не всегда может быть разумным решением - так, в случае с SVM веса перестанут являться решением задачи, поскольку нормировка изменит норму весов.

<h3 style="color:#008B8B">Все против всех (all-versus-all)</h3>

Обучим $C_K^2$ классификаторов $a_{ij}(x)$, $i, j = 1, \dots, K$, $i \neq j$.

Например, в случае с линейными моделями эти модели будут иметь вид

$\large b_k(x) = \text{sign}( \langle w_k, x \rangle + w_{0k})$

Классификатор $a_{ij}(x)$ будем настраивать по подвыборке $X_{ij} \subset X$,
содержащей только объекты классов $i$ и $j$:

$\large X_{ij} = \{(x_n, y_n) \in X | [y_n = i] = 1 \ \text{или}\ [y_n = j] = 1 \}.$

Соответственно, классификатор $a_{ij}(x)$ будет выдавать для любого объекта
либо класс $i$, либо класс $j$.

Чтобы классифицировать новый объект, подадим его на вход каждого
из построенных бинарных классификаторов.
Каждый из них проголосует за своей класс;
в качестве ответа выберем тот класс, за который наберется больше всего голосов:
$\large a(x) = \underset{k \in \{1, \dots, K\}}{\text{argmax}}
    \sum_{i = 1}^{K} \sum_{j \neq i}
        [a_{ij}(x) = k].$
        
$a(x)$ - относит $x$ к классу с наибольшим количеством побед. Каждый каласс соревнуется с каждый и какой класс победил больше всех, к тому классу мы и отнесем объект. Это долгий процесс, но он не требует калибровки.

<h2 style="color:#008B8B">1.2 Многоклассовая логистическая регрессия</h2>

Подходы выше могу быть использованы для любой модели, если необходимо решить задачу многоклассовой класификации. Но мы можем адаптировать и сами методы для многоклассовой класификации.

В логистической регрессии для двух классов мы строили линейную модель $b(x) = \langle w, x \rangle + w_0$, а затем переводили её прогноз в вероятность с помощью сигмоидной функции $\sigma(z) = \frac{1}{1 + \exp(-z)}$.

Допустим, что мы теперь решаем многоклассовую задачу и построили $K$ линейных моделей $b_k(x) = \langle w_k, x \rangle + w_{0k}$, каждая из которых даёт оценку принадлежности объекта одному из классов.

Как преобразовать вектор оценок $(b_1(x), \dots, b_K(x))$ в вероятности?
Для этого можно воспользоваться оператором $\text{SoftMax}(z_1, \dots, z_K)$, который производит "нормировку" вектора:

$\large \text{SoftMax}(z_1, \dots, z_K)
=
\left(
    \frac{\exp(z_1)}{\sum\limits_{k = 1}^{K} \exp(z_k)},
    \dots,
    \frac{\exp(z_K)}{\sum\limits_{k = 1}^{K} \exp(z_k)}
\right).$

Тем самым мы преобразовали $K$ числе в вектор, который в сумме будет равен 1 и каждый элемент вектора будет неотрицательным.
    
В этом случае вероятность $k$-го класса будет выражаться как

$\large P(y = k | x, w) =
\frac{
    \exp{(\langle w_k, x \rangle + w_{0k})}
}{
    \sum\limits_{j = 1}^{K}
        \exp{(\langle w_j, x \rangle + w_{0j})}
}.$

Обучать эти веса предлагается с помощью метода максимального правдоподобия - так же, как и в случае с двухклассовой логистической регрессией:

$\large \sum\limits_{i = 1}^{\ell}
    \log P(y = y_i | x_i, w)
\to \underset{w_1, \dots, w_K}{\text{max}}$ - мы требуем, чтобы для каждого объекта была максимальная вероятность для положитлеьного класса. Получается мы совместно обучаем все векторы всесов и $K$ линейных модели будут знать о друг друге.

**Почему мы можем интерпретировать выходы модлеи как вероятности?**

SoftMax выдает только что-то похожее на вероятности, это просто некоторое преобразование, аналогичное сигмойде. А интерпретировать как вероятности мы можем потому, что используем функцию такую фукнцию потреь, которая требует от модели корректного оценивания вероятности. Это можно показать следующим образом, если мы определение из 6 лекции обобщим на случай многоклассовой класификации, тогда для текущей фукнции потреь оно будет выполняться.

<h2 style="color:#008B8B">1.4 Метрики качества многоклассовой классификации</h2>

В многоклассовых задачах, как правило, стараются свести подсчет качества к вычислению одной из рассмотренных выше двухклассовых метрик.
Выделяют два подхода к такому сведению: микро- и макро-усреднение.

**Как посчитать precision?**

Рассмотрим задачу отделения $k$-го класса от всех остальных. Пусть выборка состоит из $K$ классов. Рассмотрим $K$ двухклассовых задач, каждая из которых заключается в отделении своего класса от остальных, то есть целевые значения для $k$-й задаче вычисляются как $y_i^k =2[y_i = k] - 1$. 

Для каждой из них можно вычислить различные характеристики (TP, FP, и т.д.) алгоритма $a^k(x) = [a(x) = k]$, которая проверяет правильно ли мы предсказали пренадлежность объекта $x$ к $k$-ому классу. Обозначим харакетирстики алгоритма следующим образом:

$\text{TP}_k, \text{FP}_k, \text{FN}_k, \text{TN}_k$ - Считаем для задачи "Класс $k$ против отсальных"

**Микро-усреднение**

При микро-усреднении сначала эти характеристики усредняются по всем классам, а затем вычисляется итоговая двухклассовая метрика - например, точность, полнота или F-мера. Например, точность будет вычисляться по формуле

$\text{precision}(a, X)
=
\frac{
    \overline{\text{TP}}
}{
    \overline{\text{TP}}
    +
    \overline{\text{FP}}
},$

где, например, $\overline{\text{TP}}$ вычисляется по формуле

$\overline{\text{TP}}
=
\frac{1}{K}
\sum\limits_{k = 1}^{K}
    \text{TP}_k.$

**Макро-усреднение**

При макро-усреднении сначала вычисляется итоговая метрика для каждого класса, а затем результаты усредняются по всем классам. Например, точность будет вычислена как

$\text{precision}(a, X)
=
\frac{1}{K}
\sum\limits_{k = 1}^{K}
    \text{precision}_k(a, X);
\qquad
\text{precision}_k(a, X)
=
\frac{
    \text{TP}_k
}{
    \text{TP}_k
    +
    \text{FP}_k
}$

**Выводы**

Если какой-то класс имеет очень маленькую мощность, то при микро-усреднении он практически никак не будет влиять на результат, поскольку его вклад в средние TP, FP, FN и TN будет незначителен. То есть, вклад класса пропорционален его размеру.

В случае же с макро-вариантом усреднение проводится для величин, которые уже не чувствительны к соотношению размеров классов (если мы используем, например, точность или полноту), и поэтому каждый класс внесет равный вклад в итоговую метрику. Это связано с тем, что precision после подсчета для любого класса принимает значение от 0 до 1.

<h1 style="color:#008B8B">2. Классификация с пересекающимися классами</h1>

Тогда пространство ответов для объекта - это всевозможные бинарные векторы размера $K$, запишем как $\mathbb{Y} = 2^K$. Класс объекта кодируется бинарным вектором длиной $K$, в котором $1$ стоят на позициях, к которым классам относится объект.

Для многоклассовой класификации мы тоже могли кодировать $y$ бинарным вектором, где $1$ могла стоять только в одной позиции из всех возможных.

Соответственно, обучающей выборке $X$ будет соответствовать матрица $Y \in \{0, 1\}^{\ell \times K}$, описывающая метки объектов; Для каждого объекта у нас $K$ записей, к какому из классов относится объект.

<h2 style="color:#008B8B">2.1 Независимая классификация (Binary relevance)</h2>

Самый простой подход к решению данной задачи предположить, что все классы независимы, и определять принадлежность объекта к каждому отдельным классификатором. Это означает, что мы обучаем $K$ бинарных классификаторов $a_1(x), \dots, a_K(x)$, причём классификатор $b_k(x)$ обучается по выборке $(x_i, y_{ik})_{i = 1}^{\ell}$. Для нового объекта $x$ целевая переменная оценивается как $(a_1(x), \dots, a_K(x))$.

<h2 style="color:#008B8B">2.2 Стекинг классификаторов</h2>

Разобьём обучающую выборку $X$ на две части $X_1$ и $X_2$. На первой части обучим $K$ независимых классификаторов $b_1(x), \dots, b_K(x)$. Далее сформируем для каждого объекта $x_i \in X_2$ из второй выборки признаковое описание, состоящее из прогнозов наших классификаторов:

$x_{i}^\prime = (b_1(x_i), \ldots, b_K(x_i)), \quad x_i \in X_2$

Тем самым, объекты второй выборки описываются оценками принадлежности к каждому из классов при помощи классификаторов прошлого шара. Получив тем самым выборку $X_2^\prime$. Обучим на ней новый набор классификаторов $a_1(x^\prime), \dots, a_K(x^\prime)$, каждый из которых определяет принадлежность объекта к одному из классов.

Новые модели уже будут знаять оценки принадлежности к каждому из классов и могу использовать эту информацию для нахождения зависимостей между между классами. Например, если публикация относится к науке, тогда шансы что этота публикация ещё относится к политике будет ниже, а к математике выше.

**Почему обучаем на разных выборках?**

Если бы мы обучали $b$ и $a$ классификаторы на одинх и тех же объектах, тогда классификаторы $b$ использовали целевую переменную для обучения и в их прогнозах содержатся подсказки о значениях целевой переменной. Значит, мы будем для классификаторов $a$ давать подсказки о целевой переменной, что понизи обобщающую способность модели.

Предположим мы построили набор классификаторов $b$ и рассмотрим первый классификатор $b_1(x)$ у которого ошибка на обучение составляет 5%, а на валидации 10%. И если мы обучим классификатор $a_1$ на выходе классификатора $b_1(x)$ для обучающей выборки, тогда $a_1$ выучит, что $b_1$ ошибается в среднем на 5% и будет использовать эту информацию для корректироваки выходов $b_1$. И когда мы для классификатора $a_1$ передадим объкты из валидации, тогда у нас будет немного другое распределение признаков $b$, так как ошибка 10%, а классификатор $a$ выучил закономерность для 5% и будет стараьтся использовать эти знания, что приведет к большей ошибке. А если обучить $a_1$ на валидации, тогда $a_1$ будет знать истенное распределение на новых данных у $b$ и качество будет лучше.